In [8]:
import google.generativeai as genai
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 

In [39]:
class Scraper:
     def scrapingData(self, url):
        DRIVER_PATH = 'C:/path/to/chromedriver/chromedriver-win64/chromedriver.exe'  
        options = Options()
        options.add_argument('--headless')
        service = Service(executable_path=DRIVER_PATH)  
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        
        elements = driver.find_elements(By.XPATH, "//h2 | //p | //ul | //i")
        content = []
        headers=[]

        for element in elements:
            text = element.text.strip() 
            if element.tag_name == 'h2' or element.tag_name == 'i':
                headers.append(text)
            if not text: 
                continue
            if element.tag_name == 'ul':
                li_elements = element.find_elements(By.TAG_NAME, 'li')
                list_items = [li.text.strip() for li in li_elements if li.text.strip()]
                if list_items:
                    content.append(" ".join(list_items))  
            else:
                content.append(text)  
        driver.quit()
        return content,headers
        

In [40]:
url = 'https://www.presight.io/privacy-policy.html'
scraper = Scraper()
content,headers = scraper.scrapingData(url)

In [43]:
def index_content(content, headers):
    indexed_content = {}
    current_index = None
    title = None
    content_list = []
    for item in content:
        # Kiểm tra nếu item có trong headers
        if item in headers:
            # Nếu có title mới, lưu lại title và content cũ
            if current_index is not None:
                indexed_content[current_index] = {
                    'title': title,
                    'content': content_list
                }   
            # Thiết lập title mới và làm mới content
            title = item
            content_list = []
            current_index = len(indexed_content)  
        else:
            # Nếu không trùng, thêm item vào content của title hiện tại
            content_list.append(item)
    
    # Lưu phần cuối cùng
    if current_index is not None:
        indexed_content[current_index] = {
            'title': title,
            'content': content_list
        }
    return indexed_content

indexed_content = index_content(content, headers)

# In kết quả
for index, data in indexed_content.items():
    print(f"Index {index}:")
    print(f"  Title: {data['title']}")
    print(f"  Content: {data['content']}")

Index 0:
  Title: PRIVACY POLICY
  Content: []
Index 1:
  Title: Last updated 15 Sep 2023
  Content: ['At Presight, we are committed to protecting the privacy of our customers and visitors to our website. This Privacy Policy explains how we collect, use, and disclose information about our customers and visitors.']
Index 2:
  Title: Information Collection and Use
  Content: ['We collect several different types of information for various purposes to provide and improve our Service to you.']
Index 3:
  Title: Types of Data Collected
  Content: []
Index 4:
  Title: Personal Data
  Content: ['While using our Service, we may ask you to provide us with certain personally identifiable information that can be used to contact or identify you ("Personal Data"). Personally identifiable information may include, but is not limited to:', 'Email address First name and last name Phone number Address, State, Province, ZIP/Postal code, City Cookies and Usage Data']
Index 5:
  Title: Usage Data
  Content: